In [1]:
import pickle
import scipy.spatial.distance
import operator
import numpy as np
import pprint
from wikidata.client import Client

In [35]:
#Wikidata client which allows to query wikidata and get a title of the passed entity, etc
client = Client()

#path to the projects with datasets and models dirs without a trailing slash
PROJECT_DIR = '/Users/alexp/wrk/kgemb'

#set the dimension of embeddings and the model used
KG_DIMENSION = 100
MODEL = 'transR' #could be transH as well ?

DATASET_NAME = 'asoif_and_hp' #or 'asoif'

#eX = X epochs, eg. e500 = 500 epochs
CONF_NAME = 'e100'

#nothing to change
ENTITY_EMBEDDING = PROJECT_DIR + '/models/' + MODEL + '/' + str(KG_DIMENSION) + '/' + DATASET_NAME + '/' + CONF_NAME + '/entity2vec.vec'
RELATION_EMBEDDING = PROJECT_DIR + '/models/' + MODEL + '/' + str(KG_DIMENSION) + '/' + DATASET_NAME + '/' + CONF_NAME + '/relation2vec.vec'
MATRIX = PROJECT_DIR + '/models/' + MODEL + '/' + str(KG_DIMENSION) + '/' + DATASET_NAME + '/' + CONF_NAME + '/A.vec'

ENTITY2ID = PROJECT_DIR + '/datasets/entity2id.txt'
RELATION2ID = PROJECT_DIR + '/datasets/relation2id.txt'
TRAIN2ID = PROJECT_DIR + '/datasets/train2id.txt'

In [25]:
names_ids = {
    'Robert': 'Q12056060',
    'Jon': 'Q3183235',
    'Robb': 'Q13634884',
    'Jaime': 'Q3806180',
    'Cersei': 'Q3665163',
    'Sansa': 'Q3472490',
    'Joffrey': 'Q12900597',
    'Tywin': 'Q12902445',
    'Tyrion': 'Q2076759',
    'Stannis': 'Q12056060',
    'Catelyn': 'Q2941743',
    'Bran': 'Q3643599',
    'Arya': 'Q3624677',
    'Ned': 'Q259818',
    'Renly': 'Q18920105',
    'Hound': 'Q3948140',
    'Varys': 'Q4008842',
    'Lysa': 'Q19791067',
    'Tommen': 'Q19792294',
#    'Pycelle': 'Maester Pycelle', #not provided in the pretrained embeddings
    'Balon': 'Q23746101',
    'Loras': 'Q18920137',
    'Theon': 'Q1120793',
    'Petyr': 'Q4360302',
    'Edmure': 'Q23749012',
    'Barristan': 'Q5721186',
    'Roose': 'Q19799430',
    'Margaery': 'Q12900933',
    'Hoster': 'Q23730371',
    'Myrcella': 'Q19799435'
}

In [26]:
def dump_entities_ids():
    """
    Save list of entities on which KG embeddings are trained
    Using the index of element in this array we can retrieve
    the coordinates of the vector from model.
    """
    entities = []
    with open(ENTITY2ID, 'r') as file:
        entities_cnt = int(file.readline())
        for _ in range(entities_cnt):
            entities.append(file.readline().split('\t')[0])
    pickle.dump(entities, open(MODEL + '_entities.pickle', 'wb'))


def dump_relations_ids():
    """
    Save list of relations on which KG embeddings are trained
    Using the index of element in this array we can retrieve
    the coordinates of the vector from model.
    """
    entities = []
    with open(RELATION2ID, 'r') as file:
        entities_cnt = int(file.readline())
        for _ in range(entities_cnt):
            entities.append(file.readline().split('\t')[0])
    pickle.dump(entities, open(MODEL + '_relation.pickle', 'wb'))


def get_entity_embedding():
    """
    Read embedding from file
    :return: numpy.memmap
    """
    return np.loadtxt(ENTITY_EMBEDDING)


def get_relation_embedding():
    """
    Read embedding from file
    :return: numpy.memmap
    """
    return np.loadtxt(RELATION_EMBEDDING)


def get_vec_from_embedding(embedding, index):
    """
    get the vector for embedding
    :param embedding: numpy.memmap object with embedding
    :param index: index of the required element
    :return:
    """
    return embedding[index]


In [27]:
# Do not have to execute if there are entities.pickle and relations.pickle already
dump_relations_ids()
dump_entities_ids()

In [36]:
entities_ids = pickle.load(open(MODEL + '_entities.pickle', 'rb'))
relation_ids = pickle.load(open(MODEL + '_relation.pickle', 'rb'))
matrix = np.loadtxt(MATRIX)

entity_embedding = get_entity_embedding()
relation_embedding = get_entity_embedding()

In [29]:
#download labels for each relation from wikidata
relation_labels = {}
for relation in relation_ids:
    relation_labels[relation] = str(client.get(relation).label)

In [37]:
tywin = get_vec_from_embedding(entity_embedding, entities_ids.index(names_ids['Tywin']))
tywin_matrix = get_vec_from_embedding(matrix, entities_ids.index(names_ids['Tywin']))
jaime = get_vec_from_embedding(entity_embedding, entities_ids.index(names_ids['Jaime']))
jaime_matrix = get_vec_from_embedding(matrix, entities_ids.index(names_ids['Jaime']))
cersei = get_vec_from_embedding(entity_embedding, entities_ids.index(names_ids['Cersei']))
cersei_matrix = get_vec_from_embedding(matrix, entities_ids.index(names_ids['Cersei']))
father = get_vec_from_embedding(relation_embedding, relation_ids.index('P22'))
mother = get_vec_from_embedding(relation_embedding, relation_ids.index('P25'))

In [38]:
# if the relation between chosen entities is correct, head+relation-tail should tend to zero
# or at least closer to zero than other relations
print(np.mean(abs(tywin*tywin_matrix+father-jaime*jaime_matrix)))
print(np.mean(abs(jaime*jaime_matrix+father-tywin*tywin_matrix)))

print(np.mean(abs(jaime*jaime_matrix+father-tywin*tywin_matrix)) > np.mean(abs(jaime*jaime_matrix+mother-tywin*tywin_matrix)))
print(np.mean(abs(tywin*tywin_matrix+father-jaime*jaime_matrix)) > np.mean(abs(tywin*tywin_matrix+mother-jaime*jaime_matrix)))

0.01285332971831
0.013120236067569999
False
False


In [39]:
# experiments with the idea provided above

rels = {}
for relation in relation_ids:
    rel = get_vec_from_embedding(relation_embedding, relation_ids.index(relation))
    #rels[relation] = np.mean(abs(tywin+rel-jaime))
    rels[relation_labels[relation]] = np.mean(abs(tywin*tywin_matrix+rel-jaime*jaime_matrix))
    
sorted_rels = sorted(rels.items(), key=operator.itemgetter(1))

print('first five items:')
pprint.pprint(sorted_rels[:5])
print()
print('last five items:')
pprint.pprint(sorted_rels[-5:])


print('\n########\n')


rels = {}
for relation in relation_ids:
    rel = get_vec_from_embedding(relation_embedding, relation_ids.index(relation))
    #rels[relation] = np.mean(abs(tywin+rel-jaime))
    rels[relation_labels[relation]] = np.mean(abs(jaime*jaime_matrix+rel-tywin*tywin_matrix))
    
sorted_rels = sorted(rels.items(), key=operator.itemgetter(1))

print('first five items:')
pprint.pprint(sorted_rels[:5])
print()
print('last five items:')
pprint.pprint(sorted_rels[-5:])

first five items:
[('spouse', 0.009233609857590001),
 ('based on', 0.009376736140089999),
 ('located in the administrative territorial entity', 0.009690378813630001),
 ('participant', 0.009745446778630002),
 ('immediate cause of', 0.01007503421457)]

last five items:
[('different from', 0.03484202285195),
 ('educated at', 0.03677833509559),
 ('office held by head of the organisation', 0.03698147279307),
 ('ethnic group', 0.037629440907570005),
 ('terminus', 0.03773791421153)]

########

first five items:
[('based on', 0.00934692380027),
 ('spouse', 0.009454669071850002),
 ('located in the administrative territorial entity', 0.009784781097409998),
 ('participant', 0.00987130234425),
 ('immediate cause of', 0.010129085785429998)]

last five items:
[('different from', 0.03470125714805),
 ('office held by head of the organisation', 0.03672333667657),
 ('educated at', 0.03683354490441),
 ('ethnic group', 0.03774086945763001),
 ('terminus', 0.037748365788470004)]


In [40]:
valid_triples = []
with open(TRAIN2ID, 'r') as file:
        triples_cnt = int(file.readline())
        for _ in range(triples_cnt):
            valid_triples.append(file.readline().split())

success = []
fail = []
changed_list = []
for index, triple in enumerate(valid_triples):
    e1 = get_vec_from_embedding(entity_embedding, int(triple[0]))
    e1_matrix = get_vec_from_embedding(matrix, int(triple[0]))
    e2 = get_vec_from_embedding(entity_embedding, int(triple[1]))
    e2_matrix = get_vec_from_embedding(matrix, int(triple[1]))
    r = get_vec_from_embedding(relation_embedding, int(triple[2]))
    r_mean = min_r = np.mean(abs(e1*e1_matrix+r-e2*e2_matrix))
    min_r_vec = r
    
    for relation in relation_ids:
        rel = get_vec_from_embedding(relation_embedding, relation_ids.index(relation))
        curr_r = np.mean(abs(e1*e1_matrix+rel-e2*e2_matrix))
        if curr_r < min_r:
            min_r = curr_r
            min_r_vec = rel
    
    
    if np.array_equal(min_r_vec,r):
        success.append(triple)
    else:
        fail.append(triple)

In [41]:
print("Successfully predicted:")
for triple in success:
    e1 = client.get(entities_ids[int(triple[0])]).label
    e2 = client.get(entities_ids[int(triple[1])]).label
    r = client.get(relation_ids[int(triple[2])]).label
    print(f'{e1} - {r} - {e2}')
    

Successfully predicted:
Eddard Stark - spouse - Catelyn Stark
Hizdahr zo Loraq - spouse - Daenerys Targaryen
Aegon I Targaryen - spouse - Visenya Targaryen
Aegon I Targaryen - spouse - Rhaenys Targaryen
Petyr Baelish - spouse - Lysa Arryn
Shaera Targaryen - spouse - Jaehaerys II Targaryen
Leyton Hightower - spouse - Rhea Florent
Betha Blackwood - spouse - Aegon V Targaryen
Rhaelle Targaryen - spouse - Ormund Baratheon
Rhea Florent - spouse - Leyton Hightower
Randyll Tarly - spouse - Melessa Tarly
Alester Florent - spouse - Melara Florent
Lynesse Hightower - spouse - Jorah Mormont
Bethany Bolton - spouse - Roose Bolton
Merrett Frey - spouse - Mariya Darry
Walda Frey - spouse - Roose Bolton
Mariya Darry - spouse - Merrett Frey
Amerei Frey - spouse - Lancel Lannister
Elia Martell - spouse - Rhaegar Targaryen
Genna Frey - spouse - Emmon Frey
Sansa Stark - spouse - Tyrion Lannister
Cersei Lannister - spouse - Robert Baratheon
Khal Drogo - spouse - Daenerys Targaryen
Stannis Baratheon - spou

Some results
===========

ASOIF and HP joined dataset
-------------------

1. 1000 Epochs
```
Successfully predicted:
Ron Weasley - member of sports team - Gryffindor quidditch team
Harry Potter - member of sports team - Gryffindor quidditch team
Jimmy Peakes - member of sports team - Gryffindor quidditch team
Angelina Johnson - member of sports team - Gryffindor quidditch team
Oliver Wood - member of sports team - Gryffindor quidditch team
Katie Bell - member of sports team - Gryffindor quidditch team
Cormac McLaggen - member of sports team - Gryffindor quidditch team
Jack Sloper - member of sports team - Gryffindor quidditch team
Ron Weasley - member of sports team - Gryffindor quidditch team
Harry Potter - member of sports team - Gryffindor quidditch team
```
2. 500 epochs
```
Successfully predicted:
Ron Weasley - member of sports team - Gryffindor quidditch team
Harry Potter - member of sports team - Gryffindor quidditch team
Jimmy Peakes - member of sports team - Gryffindor quidditch team
Angelina Johnson - member of sports team - Gryffindor quidditch team
Oliver Wood - member of sports team - Pullover United
Oliver Wood - member of sports team - Gryffindor quidditch team
Katie Bell - member of sports team - Gryffindor quidditch team
Cormac McLaggen - member of sports team - Gryffindor quidditch team
Viktor Krum - member of sports team - Bulgaria national quidditch team
Jack Sloper - member of sports team - Gryffindor quidditch team
Ron Weasley - member of sports team - Gryffindor quidditch team
Harry Potter - member of sports team - Gryffindor quidditch team
```
3. 100 epochs
```
Successfully predicted:
Eddard Stark - spouse - Catelyn Stark
Hizdahr zo Loraq - spouse - Daenerys Targaryen
Aegon I Targaryen - spouse - Visenya Targaryen
Aegon I Targaryen - spouse - Rhaenys Targaryen
Petyr Baelish - spouse - Lysa Arryn
Shaera Targaryen - spouse - Jaehaerys II Targaryen
Leyton Hightower - spouse - Rhea Florent
Betha Blackwood - spouse - Aegon V Targaryen
Rhaelle Targaryen - spouse - Ormund Baratheon
Rhea Florent - spouse - Leyton Hightower
Randyll Tarly - spouse - Melessa Tarly
Alester Florent - spouse - Melara Florent
Lynesse Hightower - spouse - Jorah Mormont
Bethany Bolton - spouse - Roose Bolton
Merrett Frey - spouse - Mariya Darry
Walda Frey - spouse - Roose Bolton
Mariya Darry - spouse - Merrett Frey
Amerei Frey - spouse - Lancel Lannister
Elia Martell - spouse - Rhaegar Targaryen
Genna Frey - spouse - Emmon Frey
Sansa Stark - spouse - Tyrion Lannister
Cersei Lannister - spouse - Robert Baratheon
Khal Drogo - spouse - Daenerys Targaryen
Stannis Baratheon - spouse - Selyse Baratheon
Daemon Targaryen - spouse - Rhaenyra Targaryen
Jeyne Poole - spouse - Ramsay Bolton
Alysanne Targaryen - spouse - Jaehaerys I Targaryen
Rhaenyra Targaryen - spouse - Daemon Targaryen
Margaery Tyrell - spouse - Joffrey Baratheon
Margaery Tyrell - spouse - Renly Baratheon
Margaery Tyrell - spouse - Tommen Baratheon
Joffrey Baratheon - spouse - Margaery Tyrell
Tywin Lannister - spouse - Joanna Lannister
Robb Stark - spouse - Talisa Stark
Robb Stark - spouse - Jeyne Westerling
Aerys II Targaryen - spouse - Rhaella Targaryen
Jon Arryn - spouse - Lysa Arryn
Ramsay Bolton - spouse - Jeyne Poole
Rhaegar Targaryen - spouse - Elia Martell
Cassana Baratheon - spouse - Steffon Baratheon
Alerie Tyrell - spouse - Mace Tyrell
Mace Tyrell - spouse - Alerie Tyrell
Balon Greyjoy - spouse - Alannys Greyjoy
Luthor Tyrell - spouse - Olenna Tyrell
Alannys Greyjoy - spouse - Balon Greyjoy
Selyse Baratheon - spouse - Stannis Baratheon
Olenna Tyrell - spouse - Luthor Tyrell
Aegon V Targaryen - spouse - Betha Blackwood
Minisa Tully - spouse - Hoster Tully
Tytos Lannister - spouse - Jeyne Lannister
Quellon Greyjoy - spouse - Lady of House Sunderly
Rickard Stark - spouse - Lyarra Stark
Lyarra Stark - spouse - Rickard Stark
Lyanna Stark - spouse - Rhaegar Targaryen
Ron Weasley - spouse - Hermione Granger
Bill Weasley - spouse - Fleur Delacour
Vernon Dursley - spouse - Petunia Dursley
Frank Longbottom - spouse - Alice Longbottom
Harry Potter - spouse - Ginny Weasley
Merope Gaunt - spouse - Tom Riddle
Astoria Greengrass - spouse - Draco Malfoy
Tobias Snape - spouse - Eileen Prince
Luna Lovegood - spouse - Rolf Scamander
Tom Riddle - spouse - Merope Gaunt
Remus Lupin - spouse - Nymphadora Tonks
Kendra Dumbledore - spouse - Percival Dumbledore
Neville Longbottom - spouse - Hannah Abbott
Pollux Black - spouse - Irma Crabbe
George Weasley - spouse - Angelina Johnson
Draco Malfoy - spouse - Astoria Greengrass
Ignatius Prewett - spouse - Lucretia Black
Andromeda Tonks - spouse - Ted Tonks
Mosag - spouse - Aragog
Molly Weasley - spouse - Arthur Weasley
Bellatrix Lestrange - spouse - Rodolphus Lestrange
Fleur Delacour - spouse - Bill Weasley
Lily Potter - spouse - James Potter
Dorea Black - spouse - Charlus Potter
Sirius Black II - spouse - Hesper Gamp
Herbert Burke - spouse - Belvina Black
Druella Rosier - spouse - Cygnus Black III
Bob Hitchens - spouse - Isla Black
Lucretia Black - spouse - Ignatius Prewett
Arcturus Black III - spouse - Melania Macmillan
Hesper Gamp - spouse - Sirius Black II
Mrs. Evans - spouse - Mr. Evans
Mr. Evans - spouse - Mrs. Evans
Septimus Weasley - spouse - Cedrella Black
Cedrella Black - spouse - Septimus Weasley
Callidora Black - spouse - Harfang Longbottom
Caspar Crouch - spouse - Charis Black
Arcturus Black - spouse - Lysandra Yaxley
Cygnus Black - spouse - Violetta Bulstrode
Belvina Black - spouse - Herbert Burke
Phineas Nigellus Black - spouse - Ursula Flint
Lucius Malfoy - spouse - Narcissa Malfoy
Orion Black - spouse - Walburga Black
Violetta Bulstrode - spouse - Cygnus Black
Alice Longbottom - spouse - Frank Longbottom
Eileen Prince - spouse - Tobias Snape
Ted Tonks - spouse - Andromeda Tonks
Nymphadora Tonks - spouse - Remus Lupin
Ron Weasley - spouse - Hermione Granger
Luna Lovegood - spouse - Rolf Scamander
Neville Longbottom - spouse - Hannah Abbott
Lucius Malfoy - spouse - Narcissa Malfoy
Draco Malfoy - spouse - Astoria Greengrass
Bellatrix Lestrange - spouse - Rodolphus Lestrange
Lily Potter - spouse - James Potter
Harry Potter - spouse - Ginny Weasley
Vernon Dursley - spouse - Petunia Dursley
```
4. 50 epochs
```
Successfully predicted:
Eddard Stark - spouse - Catelyn Stark
Emmon Frey - spouse - Genna Frey
Tyrion Lannister - spouse - Sansa Stark
Daenerys Targaryen - spouse - Khal Drogo
Daenerys Targaryen - spouse - Hizdahr zo Loraq
Hizdahr zo Loraq - spouse - Daenerys Targaryen
Catelyn Stark - spouse - Eddard Stark
Aegon I Targaryen - spouse - Visenya Targaryen
Aegon I Targaryen - spouse - Rhaenys Targaryen
Jorah Mormont - spouse - Lynesse Hightower
Petyr Baelish - spouse - Lysa Arryn
Shaera Targaryen - spouse - Jaehaerys II Targaryen
Ormund Baratheon - spouse - Rhaelle Targaryen
Leyton Hightower - spouse - Rhea Florent
Betha Blackwood - spouse - Aegon V Targaryen
Rhaelle Targaryen - spouse - Ormund Baratheon
Jaehaerys II Targaryen - spouse - Shaera Targaryen
Rhea Florent - spouse - Leyton Hightower
Randyll Tarly - spouse - Melessa Tarly
Melessa Tarly - spouse - Randyll Tarly
Alester Florent - spouse - Melara Florent
Melara Florent - spouse - Alester Florent
Lynesse Hightower - spouse - Jorah Mormont
Bethany Bolton - spouse - Roose Bolton
Merrett Frey - spouse - Mariya Darry
Walda Frey - spouse - Roose Bolton
Mariya Darry - spouse - Merrett Frey
Amerei Frey - spouse - Lancel Lannister
Lancel Lannister - spouse - Amerei Frey
Elia Martell - spouse - Rhaegar Targaryen
Genna Frey - spouse - Emmon Frey
Sansa Stark - spouse - Tyrion Lannister
Cersei Lannister - spouse - Robert Baratheon
Khal Drogo - spouse - Daenerys Targaryen
Stannis Baratheon - spouse - Selyse Baratheon
Daemon Targaryen - spouse - Rhaenyra Targaryen
Jeyne Poole - spouse - Ramsay Bolton
Jaehaerys I Targaryen - spouse - Alysanne Targaryen
Alysanne Targaryen - spouse - Jaehaerys I Targaryen
Jeyne Westerling - spouse - Robb Stark
Rhaenys Targaryen - spouse - Aegon I Targaryen
Visenya Targaryen - spouse - Aegon I Targaryen
Rhaenyra Targaryen - spouse - Daemon Targaryen
Margaery Tyrell - spouse - Joffrey Baratheon
Margaery Tyrell - spouse - Renly Baratheon
Margaery Tyrell - spouse - Tommen Baratheon
Joffrey Baratheon - spouse - Margaery Tyrell
Tywin Lannister - spouse - Joanna Lannister
Robb Stark - spouse - Talisa Stark
Robb Stark - spouse - Jeyne Westerling
Robert Baratheon - spouse - Cersei Lannister
Renly Baratheon - spouse - Margaery Tyrell
Aerys II Targaryen - spouse - Rhaella Targaryen
Lysa Arryn - spouse - Petyr Baelish
Lysa Arryn - spouse - Jon Arryn
Jon Arryn - spouse - Lysa Arryn
Tommen Baratheon - spouse - Margaery Tyrell
Ramsay Bolton - spouse - Jeyne Poole
Roose Bolton - spouse - Bethany Bolton
Roose Bolton - spouse - Walda Frey
Rhaegar Targaryen - spouse - Lyanna Stark
Rhaegar Targaryen - spouse - Elia Martell
Joanna Lannister - spouse - Tywin Lannister
Cassana Baratheon - spouse - Steffon Baratheon
Hoster Tully - spouse - Minisa Tully
Steffon Baratheon - spouse - Cassana Baratheon
Rhaella Targaryen - spouse - Aerys II Targaryen
Alerie Tyrell - spouse - Mace Tyrell
Mace Tyrell - spouse - Alerie Tyrell
Balon Greyjoy - spouse - Alannys Greyjoy
Luthor Tyrell - spouse - Olenna Tyrell
Alannys Greyjoy - spouse - Balon Greyjoy
Selyse Baratheon - spouse - Stannis Baratheon
Olenna Tyrell - spouse - Luthor Tyrell
Aegon V Targaryen - spouse - Betha Blackwood
Minisa Tully - spouse - Hoster Tully
Jeyne Lannister - spouse - Tytos Lannister
Tytos Lannister - spouse - Jeyne Lannister
Lady of House Sunderly - spouse - Quellon Greyjoy
Quellon Greyjoy - spouse - Lady of House Sunderly
Rickard Stark - spouse - Lyarra Stark
Lyarra Stark - spouse - Rickard Stark
Lyanna Stark - spouse - Rhaegar Targaryen
Hermione Granger - spouse - Ron Weasley
Ron Weasley - spouse - Hermione Granger
Bill Weasley - spouse - Fleur Delacour
Rodolphus Lestrange - spouse - Bellatrix Lestrange
Vernon Dursley - spouse - Petunia Dursley
Frank Longbottom - spouse - Alice Longbottom
Harry Potter - spouse - Ginny Weasley
Merope Gaunt - spouse - Tom Riddle
Astoria Greengrass - spouse - Draco Malfoy
Rolf Scamander - spouse - Luna Lovegood
Petunia Dursley - spouse - Vernon Dursley
Tobias Snape - spouse - Eileen Prince
Angelina Johnson - spouse - George Weasley
Luna Lovegood - spouse - Rolf Scamander
Tom Riddle - spouse - Merope Gaunt
Remus Lupin - spouse - Nymphadora Tonks
Hannah Abbott - spouse - Neville Longbottom
Kendra Dumbledore - spouse - Percival Dumbledore
Neville Longbottom - spouse - Hannah Abbott
Pollux Black - spouse - Irma Crabbe
George Weasley - spouse - Angelina Johnson
Draco Malfoy - spouse - Astoria Greengrass
Ignatius Prewett - spouse - Lucretia Black
Andromeda Tonks - spouse - Ted Tonks
Mosag - spouse - Aragog
Molly Weasley - spouse - Arthur Weasley
Ginny Weasley - spouse - Harry Potter
Bellatrix Lestrange - spouse - Rodolphus Lestrange
Fleur Delacour - spouse - Bill Weasley
James Potter - spouse - Lily Potter
Lily Potter - spouse - James Potter
Narcissa Malfoy - spouse - Lucius Malfoy
Charlus Potter - spouse - Dorea Black
Dorea Black - spouse - Charlus Potter
Sirius Black II - spouse - Hesper Gamp
Isla Black - spouse - Bob Hitchens
Herbert Burke - spouse - Belvina Black
Cygnus Black III - spouse - Druella Rosier
Druella Rosier - spouse - Cygnus Black III
Bob Hitchens - spouse - Isla Black
Lucretia Black - spouse - Ignatius Prewett
Arcturus Black III - spouse - Melania Macmillan
Melania Macmillan - spouse - Arcturus Black III
Hesper Gamp - spouse - Sirius Black II
Mrs. Evans - spouse - Mr. Evans
Mr. Evans - spouse - Mrs. Evans
Septimus Weasley - spouse - Cedrella Black
Cedrella Black - spouse - Septimus Weasley
Harfang Longbottom - spouse - Callidora Black
Callidora Black - spouse - Harfang Longbottom
Charis Black - spouse - Caspar Crouch
Caspar Crouch - spouse - Charis Black
Arcturus Black - spouse - Lysandra Yaxley
Lysandra Yaxley - spouse - Arcturus Black
Ursula Flint - spouse - Phineas Nigellus Black
Cygnus Black - spouse - Violetta Bulstrode
Belvina Black - spouse - Herbert Burke
Phineas Nigellus Black - spouse - Ursula Flint
Lucius Malfoy - spouse - Narcissa Malfoy
Percival Dumbledore - spouse - Kendra Dumbledore
Orion Black - spouse - Walburga Black
Walburga Black - spouse - Orion Black
Irma Crabbe - spouse - Pollux Black
Violetta Bulstrode - spouse - Cygnus Black
Alice Longbottom - spouse - Frank Longbottom
Eileen Prince - spouse - Tobias Snape
Arthur Weasley - spouse - Molly Weasley
Ted Tonks - spouse - Andromeda Tonks
Nymphadora Tonks - spouse - Remus Lupin
Hermione Granger - spouse - Ron Weasley
Ron Weasley - spouse - Hermione Granger
Luna Lovegood - spouse - Rolf Scamander
Ginny Weasley - spouse - Harry Potter
Neville Longbottom - spouse - Hannah Abbott
James Potter - spouse - Lily Potter
Lucius Malfoy - spouse - Narcissa Malfoy
Draco Malfoy - spouse - Astoria Greengrass
Bellatrix Lestrange - spouse - Rodolphus Lestrange
Lily Potter - spouse - James Potter
Harry Potter - spouse - Ginny Weasley
Vernon Dursley - spouse - Petunia Dursley```